In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import warnings
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('Epileptic_Seizure_Recognition.csv')
df


,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,...,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,...,121,135,148,143,116,86,68,59,55,3


In [10]:
print(df.shape)

(11500, 180)


### Feature Label extraction


In [11]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1:]

In [12]:
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
1,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,20,15,16,12,5,-1,-18,-37,-47,-48
11496,-47,-11,28,77,141,211,246,240,193,136,...,-94,-65,-33,-7,14,27,48,77,117,170
11497,14,6,-13,-16,10,26,27,-9,4,14,...,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8
11498,-40,-25,-9,-12,-2,12,7,19,22,29,...,114,121,135,148,143,116,86,68,59,55


In [13]:
y

,y
0,4
1,1
2,5
3,5
4,5
...,...
11495,2
11496,1
11497,5
11498,3


### Feature Engineering
#### Multi label to binary label conversion
#### All subjects falling in classes 2, 3, 4, and 5 are subjects who did not have epileptic seizure. Only subjects in class 1 have epileptic seizure.

In [14]:
def toBinary(x):
    if x != 1: return 0;
    else: return 1;

In [15]:
y = y['y'].apply(toBinary)
y = pd.DataFrame(data=y)


In [16]:
y

,y
0,0
1,1
2,0
3,0
4,0
...,...
11495,0
11496,1
11497,0
11498,0


### Splitting Data (Train-Test)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)

In [18]:
# create a Logistic LogisticRegression model and fit it to the data and take new predictions
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
# print the pridction
print(y_pred)


[0 1 0 ... 1 0 0]


In [57]:
# print the data from the input.csv file
# read the data first and then print it
df = pd.read_csv('input.csv')

row=df.iloc[3,1:-1]


# change the dtpye of the data to int64
row=row.astype('int64')
row



X1     -278
X2     -246
X3     -215
X4     -191
X5     -177
       ... 
X174   -226
X175   -174
X176   -125
X177    -79
X178    -40
Name: 3, Length: 178, dtype: int64

In [58]:
# reshape the row to be able to predict it
row = np.array(row).reshape(1, -1)
# print the new shape
print(row.shape)
# predict the row
y_pred = model.predict(row)
# print the prediction
print(y_pred)

(1, 178)
[0]


In [33]:
# inp=int(input("Enter the number of rows: "))



In [54]:
# row = x_test.iloc[0]
# print the row
# print(type(row))
print(row)

# reshape the row to be able to predict it
row = np.array(row).reshape(1, -1)
# rshape argurmenst?

# print the new shape
print(row.shape)
# predict the row
y_pred = model.predict(row)
# print the prediction
print(y_pred)

X1      -5
X2     -13
X3      -4
X4       3
X5       4
        ..
X174   -17
X175   -18
X176   -10
X177   -10
X178   -15
Name: 5556, Length: 178, dtype: int64
(1, 178)
[0]
